In [1]:
import tensorflow as tf
from tensorflow.keras.models import clone_model
from model_utils import simplecnn
import numpy as np

In [2]:
class clint_k():
    def __init__(self, name, dataset, model, optimizer=None):
        self.name = name
        self.dataset = dataset
        self.attri = {'name':self.name, 'dataset':self.dataset}
        self.model = model 
    def set_params(self, weights): # input weight
        self.model.set_weights(weights)
        
    def local_update(self, optimizer =  tf.keras.optimizers.Adam(learning_rate=1e-3),
                     loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                     epochs = 1
                    ):
#         assert model!=None, 'please use set_params method first'
        optimizer = optimizer
        loss_fn = loss_fn
        model = self.model
        model.compile(optimizer, loss_fn)
        return model.fit(self.dataset, epochs=epochs)
        


In [3]:
# class CustomModel(tf.keras.Model):
#     def train_step(self, data):
#         # Unpack the data. Its structure depends on your model and
#         # on what you pass to `fit()`.
#         x, y = data

#         with tf.GradientTape() as tape:
#             y_pred = self(x, training=True)  # Forward pass
#             # Compute the loss value
#             # (the loss function is configured in `compile()`)
#             loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)

#         # Compute gradients
#         trainable_vars = self.trainable_variables
#         gradients = tape.gradient(loss, trainable_vars)
#         # Update weights
#         self.optimizer.apply_gradients(zip(gradients, trainable_vars))
#         # Update metrics (includes the metric that tracks the loss)
#         self.compiled_metrics.update_state(y, y_pred)
#         # Return a dict mapping metric names to current value
        
#         self.get_grads = [g.numpy() for g in gradients]
#         return {m.name: m.result() for m in self.metrics}

# z.model.get_grads

In [4]:
model = simplecnn()
type(model) 

tensorflow.python.keras.engine.functional.Functional

In [5]:
# model = CustomModel(model.inputs, model.outputs)

In [6]:
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test_origin, y_test_origin) = mnist.load_data()

X_train = X_train/255.0

In [7]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).cache().batch(64).prefetch(1)

In [8]:
z = clint_k(name='id0', dataset=train_dataset, model=model)
z.set_params(model.get_weights())
z.model

In [9]:
# z.local_update()

In [11]:
class PerturbedGradientDescent(tf.keras.optimizers.SGD):
    def __init__(self,
              name="PGD",
              mu=0.01,
               **kwargs):
        super().__init__(name = name, **kwargs)
#         print(name)
#         print(self._momentum)
#         print(self.nesterov)
        self.mu = mu
        self._set_hyper("prox_mu", self.mu)
        # Tensor versions of the constructor arguments, created in _prepare().
        self._lr_t = None
        self._mu_t = None
        
        self._lr_t = tf.convert_to_tensor(self._hyper['learning_rate'], name="learning_rate")
        self._mu_t = tf.convert_to_tensor(self.mu, name="prox_mu")
#     def _prepare_local(self, **kwargs):
        
    def _create_slots(self, var_list):
#         super()._create_slots(var_device, var_dtype, apply_state)
        
        for var in var_list:
            self.add_slot(var, "vstar")
#             super().__init__(**kwargs)
    
    def _resource_apply_dense(self, grad, var):
        lr_t = tf.cast(self._lr_t, var.dtype.base_dtype)
        mu_t = tf.cast(self._mu_t, var.dtype.base_dtype)
        vstar = self.get_slot(var, "vstar")
        
        var_update = var.assign_sub(lr_t*(grad + mu_t*(var-vstar)))

#         return tf.group(*[var_update,])
        return tf.raw_ops.ResourceApplyGradientDescent(
          var=var_update.handle,
          alpha=lr_t,
          delta=grad,
          use_locking=self._use_locking)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "learning_rate": self._serialize_hyperparameter("learning_rate"),
            "decay": self._initial_decay,
            "momentum": self._serialize_hyperparameter("momentum"),
            "nesterov": self.nesterov,
            "prox_mu": self.mu,
        })
        return config

In [12]:
a = PerturbedGradientDescent(nesterov=True)
a.get_config()
# a._hyper

{'name': 'PGD',
 'learning_rate': 0.01,
 'decay': 0.0,
 'momentum': 0.0,
 'nesterov': True,
 'prox_mu': 0.01}

In [13]:
z = clint_k(name='id0', dataset=train_dataset, model=model)
z.set_params(model.get_weights())

optimizer = PerturbedGradientDescent()
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    
    
z.local_update(optimizer = optimizer, loss_fn=loss_fn, )

938/938 [==============================] - 3s 3ms/step - loss: 2.2419
